In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm import tqdm

In [2]:
class Particle:
    
    def __init__(self, r0, v0, a0, t, m=1, radius=2., Id=0):
        
        self.dt = t[1] - t[0]
        
        
        # Atributos instantaneos
        self.r = r0
        self.v = v0
        self.a = a0
        
        self.m = m
        self.radius = radius
        self.Id = Id
        
        self.p = self.m*self.v
        
        # Historial
        
        self.R = np.zeros((len(t),len(r0)))
        self.V = np.zeros_like(self.R)
        self.A = np.zeros_like(self.R)
        
        self.F = np.zeros_like(self.R)
        
        self.P = np.zeros_like(self.R)
    
        # Fisica
        self.K = 10.
        
    def Evolution(self,i):
        
        
        self.r += self.dt*self.v
        self.v += self.dt*self.a
        
    # Setter
    def SetPosition(self,i):
        self.R[i] = self.r
    
    # Getter
    def GetPosition(self,scale=1):
        return self.R[::scale]

In [ ]:
def GetParticle()